In [1]:
import os
import json
import numpy as np
import cv2
import glob
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask
import torch
from torch.utils.data import Dataset, DataLoader
import random
from common import *
import re
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import itertools

In [2]:
def convert_coco_poly_to_mask(segmentations, height, width):
    masks = []
    for polygons in segmentations:
        if isinstance(polygons['counts'], list):
            rles = coco_mask.frPyObjects(polygons, height, width)
        
        else:
            rles = [polygons]

        mask = coco_mask.decode(rles)
        if len(mask.shape) < 3:
            mask = mask[..., None]
        #mask = torch.as_tensor(mask, dtype=torch.uint8)
        #mask = mask.any(dim=2)
        masks.append(mask)
    # if masks:
    #     masks = torch.stack(masks, dim=0)
    # else:
    #     masks = torch.zeros((0, height, width), dtype=torch.uint8)
    return masks

In [3]:
annotation_indices = [28, 33, 42, 59]

In [4]:
# file_path = f"/storage/user/brasoand/motsyn2/annotations/{str(idx).zfill(3)}.json"
# with open(file_path) as fp:
#     annotation_19 = json.load(fp)
    
# file_path_28 = "/storage/user/brasoand/motsyn2/annotations/028.json"
# with open(file_path_28) as fp:
#     annotation_28 = json.load(fp)

In [5]:
annotations_list = []
for idx in annotation_indices:
    file_path = f"/storage/user/brasoand/motsyn2/annotations/{str(idx).zfill(3)}.json"
    with open(file_path) as fp:
        annotations_list.append(json.load(fp))

In [6]:
annotations_list[0]['annotations'][0].keys()

dict_keys(['id', 'image_id', 'category_id', 'segmentation', 'area', 'bbox', 'iscrowd', 'keypoints', 'keypoints_3d', 'num_keypoints', 'ped_id', 'model_id', 'attributes', 'frame_n', 'seq_name', 'is_night', 'weather', 'rec_time', 'is_moving'])

In [7]:
annotations_list[0].keys()

dict_keys(['info', 'licences', 'images', 'annotations', 'categories'])

In [8]:
for annotation in annotations_list:
    for idx, ann in enumerate(annotation['annotations']):
        try:
            ann['video_id'] = ann.pop('image_id')
            masks = convert_coco_poly_to_mask([ann['segmentation']], 1920, 1080)
            ann['segmentation'] = coco_mask.encode(masks[0][:,0:1080])[0]
        except Exception as e:
            print(ann)
            raise e

In [9]:
for annotation in annotations_list:
    annotation['annotations'] = [ann for ann in annotation['annotations'] if ann["video_id"] % 5 == 0]
    annotation['annotations'] = [ann for ann in annotation['annotations'] if ann["category_id"] == 1]

In [10]:
backup_annotations_list = annotations_list.copy()
#

In [ ]:
# backup_annotations_list = annotations_list.copy()
annotations_list = backup_annotations_list.copy()

In [ ]:
type(annotations_list[0]["annotations"][0]["keypoints"])

In [ ]:
annotations_list[0]["annotations"][0]["bbox"]

In [11]:
evaluation_annotation_file = {}
evaluation_annotation_file['info'] = {'description': 'MOTSynth 2021 Dataset - Sequence #59',
                                     'url': 'http://aimagelab.ing.unimore.it/jta',
                                     'version': '1.0',
                                     'year': 2021,
                                     'contributor': 'AImageLab',
                                     'date_created': '2021/01/22'}
evaluation_annotation_file['videos'] = []
evaluation_annotation_file['annotations'] = []
evaluation_annotation_file['categories'] = [{'supercategory': 'object', 'id': 1, 'name': 'person'}]


none_mask = np.zeros((1080,1080), dtype=np.uint8)
none_mask = coco_mask.encode(np.asfortranarray(none_mask))
none_mask["counts"] = none_mask["counts"].decode("utf-8")


for a_idx, annotation in enumerate(annotations_list):
    annotation["videos"] = []
    ped_ids = list(set([ann["ped_id"] for ann in annotation['annotations'] if ann["video_id"] % 5 == 0]))
    videos_dict = {"id" : int(annotation['images'][0]['file_name'].split("/")[1]),
                   "width" : 1080,
                   "height" : 1080,
                   "length" : 360,
                   "file_names" : [img['file_name'] for img in annotation['images'] if int(img['file_name'].split("/")[-1].split(".")[0]) % 5 == 0]}
    
    
    annotations_dict = [{"id" : int,
                   "video_id" : int,
                   "category_id" : int,
                   "iscrowd" : int,
                   "areas" : [None for i in range(360)],
                   "segmentations": [none_mask.copy() for i in range(360)],
                   "bboxes": [None for i in range(360)]} for x in ped_ids]
    for idx, ann in enumerate(annotation['annotations']):
        current_ped_id = ann["ped_id"]
        current_ped_id_idx = ped_ids.index(current_ped_id)
        current_frame_no = int(int(ann["video_id"] % 10000) / 5)
        current_segmentation = ann["segmentation"]
        current_segmentation["counts"] = current_segmentation["counts"].decode("utf-8")
        current_category_id = ann["category_id"]
        current_area = ann["area"]
        current_bbox = ann["bbox"]
        current_iscrowd = ann["iscrowd"]
        annotations_dict[current_ped_id_idx]["id"] = current_ped_id
        annotations_dict[current_ped_id_idx]["areas"][current_frame_no] = current_area
        annotations_dict[current_ped_id_idx]["bboxes"][current_frame_no] = current_bbox
        annotations_dict[current_ped_id_idx]["segmentations"][current_frame_no] = current_segmentation
        annotations_dict[current_ped_id_idx]["category_id"] = current_category_id
        annotations_dict[current_ped_id_idx]["video_id"] = int(ann["video_id"] // 10000)
        annotations_dict[current_ped_id_idx]["iscrowd"] = current_iscrowd
        
    evaluation_annotation_file['videos'].append(videos_dict)
    print(len(annotations_dict))
    evaluation_annotation_file['annotations'].extend(annotations_dict)


77
72
96
55


In [ ]:
dict(segmentation)

In [ ]:
evaluation_annotation_file["annotations"] = [ann for ann in evaluation_annotation_file["annotations"]]

In [ ]:
# for annotation in evaluation_annotation_file["annotations"]:
#     for segmentation in annotation["segmentations"]:
#         segmentation["counts"] = segmentation["counts"].decode("utf-8")

In [13]:
evaluation_annotation_file.keys()

dict_keys(['info', 'videos', 'annotations', 'categories'])

In [12]:
import json
with open('evaluation/gt_data_eval.json', 'w') as f:
    json.dump(evaluation_annotation_file, f)

In [1]:
annotation_indices = [28, 33, 42, 59]
video_indice = -1
seq_id = 0
predictions_list = []
for i in range(1800, (1 + len(annotation_indices)) * 36):
    print(i)

In [2]:
36*19

684